In [1]:
!pip install torchvision
!pip install pandas
!pip install -U scikit-learn
!pip install seaborn
!pip install missingno
!pip install preprocessing

In [2]:
# Cell 1: Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import csv
from sklearn.metrics import fbeta_score
from sklearn.metrics import confusion_matrix
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno # visualizing missing values
import seaborn as sns # for data visualization
import matplotlib.pyplot as plt # for plot
from datetime import datetime
from datetime import time
%matplotlib inline

import torch 
import torch.nn as nn # For neural network building
import torch.nn.functional as F # For activation function
from torch.utils import data  # For data handling
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler # Same as previous
from sklearn.model_selection import train_test_split # For splitting the data
from sklearn.model_selection import KFold # For kfold training
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score

In [3]:
original_df = pd.read_csv('Day2/dataset2/NextGen Regression/test_regress.csv')
new_df = pd.read_csv('Day2/old submission/waroon_submission(9).csv')

original_df['3D'] = new_df['3D']
original_df['symmetric'] = new_df['symmetric']
original_df['food'] = new_df['food']
original_df['person'] = new_df['person']
original_df['nature'] = new_df['nature']
original_df['architectural'] = new_df['architectural']
original_df['animal'] = new_df['animal']

season_list = []
hour_list = []
minute_list = []
for i in range(len(original_df)):
    date = original_df['date'][i].split("/")[1]
    time = original_df['date'][i].split(" ")[1]
    hour,minute = time.split(":")
    if(int(date) <= 3):
        season_list.append(1)
    elif(int(date) <= 6):
        season_list.append(2)
    elif(int(date) <= 9):
        season_list.append(3)
    elif (int(date) <= 12):
        season_list.append(4)
    hour_list.append(int(hour))
    minute_list.append(int(minute))

original_df['season'] = season_list
original_df['hour'] = hour_list
original_df['minute'] = minute_list

original_df.to_csv("Day2/dataset2/NextGen Regression/test_regress.csv",index=False)

In [4]:
# set default device based on CUDA's availability 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
df = pd.read_csv(r'Day2/dataset2/NextGen Regression/train_regress.csv')

In [6]:
x = np.asarray(df[['3D','symmetric','food','person','architectural','animal','nature','bargain','inflation','grade']])
y = np.asarray(df['value'])

In [7]:
x = preprocessing.StandardScaler().fit(x).transform(x)

In [8]:
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=4)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (2310, 10) (2310,)
Test set: (578, 10) (578,)


In [9]:
class NN(nn.Module):
  def __init__(self):
    super(NN,self).__init__()
    self.layer1=nn.Linear(x_train.shape[1],128)
    self.layer2=nn.Linear(128,64)
    self.layer3=nn.Linear(64,32)
    self.layer4=nn.Linear(32,12)
    self.layer5=nn.Linear(12,6)
    self.layer6=nn.Linear(6,1)
  def forward(self,x):
    x=F.relu(self.layer1(x))
    x=F.relu(self.layer2(x))
    x=F.relu(self.layer3(x))
    x=F.relu(self.layer4(x))
    x=F.relu(self.layer5(x))
    x=self.layer6(x)
    return x
model = NN()
print(model)

NN(
  (layer1): Linear(in_features=10, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=32, bias=True)
  (layer4): Linear(in_features=32, out_features=12, bias=True)
  (layer5): Linear(in_features=12, out_features=6, bias=True)
  (layer6): Linear(in_features=6, out_features=1, bias=True)
)


In [10]:
x=torch.tensor(np.array(x_train),dtype=torch.float32,requires_grad=True)
y=torch.tensor(np.array(y_train).reshape(-1,1),dtype=torch.float32)
print(x.shape)
print(x)

torch.Size([2310, 10])


tensor([[ 0.5185, -0.2055, -0.2036,  ..., -1.2896,  1.1929,  0.2648],
        [ 0.5185, -0.2055, -0.2036,  ...,  0.0132, -1.6664, -1.3213],
        [ 0.5185, -0.2055, -0.2036,  ...,  0.2830,  0.0067, -1.2279],
        ...,
        [ 0.5185, -0.2055, -0.2036,  ...,  0.5492,  1.4084, -1.2191],
        [ 0.5185, -0.2055, -0.2036,  ..., -1.5418,  1.5870,  0.2736],
        [ 0.5185, -0.2055, -0.2036,  ..., -0.9321, -1.3055, -1.5352]],
       requires_grad=True)


In [11]:
optimizer= optim.Adam(model.parameters(),lr=0.003)

In [12]:
loss_fn=nn.MSELoss()

In [13]:
epochs = 10000
for i in range(epochs):
  #initialize the model parameter
  optimizer.zero_grad(set_to_none=True)
  #calculate the loss
  output=model(x)
  loss=loss_fn(output,y)
  #backpropagation
  loss.backward()
  #update the parameters
  optimizer.step()
  if(i%5==0):
    print(f"epochs: {i}......loss:{loss}")

epochs: 0......loss:7415041.0
epochs: 5......loss:7414698.0
epochs: 10......loss:7413484.5
epochs: 15......loss:7409726.5
epochs: 20......loss:7399241.5
epochs: 25......loss:7372627.5
epochs: 30......loss:7310332.0
epochs: 35......loss:7174534.0
epochs: 40......loss:6896771.5
epochs: 45......loss:6365397.0
epochs: 50......loss:5425747.0
epochs: 55......loss:3940379.0
epochs: 60......loss:2027903.875
epochs: 65......loss:626685.125
epochs: 70......loss:914838.1875
epochs: 75......loss:561537.25
epochs: 80......loss:286030.8125
epochs: 85......loss:329113.875
epochs: 90......loss:252119.796875
epochs: 95......loss:175386.703125
epochs: 100......loss:178474.8125
epochs: 105......loss:154731.828125
epochs: 110......loss:134233.9375
epochs: 115......loss:130436.75
epochs: 120......loss:120024.2734375
epochs: 125......loss:112982.3671875
epochs: 130......loss:108966.5859375
epochs: 135......loss:103717.4140625
epochs: 140......loss:100215.4375
epochs: 145......loss:97147.859375
epochs: 150..

In [14]:
y_train_pred = model(torch.tensor(x_train,dtype=torch.float32,requires_grad=True))
y_test_pred = model(torch.tensor(x_test,dtype=torch.float32))

#convert to numpy array
y_train_pred = y_train_pred.detach().numpy()
y_test_pred = y_test_pred.detach().numpy()

In [15]:
print(y_test_pred)

[[3168.6824 ]
 [2635.0977 ]
 [3908.1484 ]
 [3855.7134 ]
 [1496.939  ]
 [3180.2583 ]
 [3073.9382 ]
 [3166.9346 ]
 [3013.8213 ]
 [4267.401  ]
 [3181.6475 ]
 [3160.3574 ]
 [1725.0529 ]
 [4541.2563 ]
 [2797.3345 ]
 [2082.444  ]
 [2597.3945 ]
 [2631.0386 ]
 [3595.4724 ]
 [2712.0703 ]
 [3206.3877 ]
 [3113.4856 ]
 [1116.0359 ]
 [3218.501  ]
 [3134.1074 ]
 [2562.4702 ]
 [3204.9854 ]
 [3179.9995 ]
 [2599.859  ]
 [1647.3936 ]
 [3092.7793 ]
 [2758.4922 ]
 [1622.4777 ]
 [2651.8745 ]
 [3251.246  ]
 [1068.3279 ]
 [3133.4404 ]
 [1065.249  ]
 [1675.0253 ]
 [3230.8706 ]
 [3064.4614 ]
 [ 985.5816 ]
 [1537.1978 ]
 [4188.1377 ]
 [1630.9194 ]
 [3259.9136 ]
 [1870.5089 ]
 [2588.56   ]
 [2677.9126 ]
 [1205.7216 ]
 [2599.486  ]
 [3132.4722 ]
 [2650.882  ]
 [3570.7402 ]
 [2755.0413 ]
 [2793.9932 ]
 [3714.4922 ]
 [3022.753  ]
 [3310.6196 ]
 [3069.4375 ]
 [2295.7734 ]
 [3148.5122 ]
 [1080.3757 ]
 [4090.108  ]
 [3235.5776 ]
 [2724.463  ]
 [1255.6627 ]
 [2981.4158 ]
 [2741.53   ]
 [2648.153  ]
 [2678.1836 ]
 [2916

In [16]:
test_accuracy=r2_score(y_test,y_test_pred)
train_accuracy=r2_score(y_train,y_train_pred)
print(train_accuracy)
print(test_accuracy)

0.952689422923309
0.9006644953587951


In [17]:
torch.save(model.state_dict(), 'Day2/weight/weight.pth')
print("Model saved successfully")

Model saved successfully


In [18]:
test_dataset_df = pd.read_csv('Day2/dataset2/NextGen Regression/test_regress.csv')
test_dataset_df

,ID,3D,symmetric,food,person,architectural,animal,nature,bargain,inflation,grade,date,season,hour,minute
0,1,1,0,0,0,0,0,1,0.633198,0.978743,0.859228,1/1/2024 21:32,1,21,32
1,2,0,0,0,1,0,0,0,0.544848,0.985882,1.040116,2/1/2024 16:58,1,16,58
2,3,0,0,0,1,0,0,0,0.691743,0.988934,1.158181,3/1/2024 18:41,1,18,41
3,4,1,0,0,0,0,0,0,0.639707,0.981185,0.944608,4/1/2024 3:53,1,3,53
4,5,1,0,0,1,0,0,0,0.757101,0.984028,0.898883,5/1/2024 0:32,1,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,719,1,0,0,0,0,0,1,0.799537,0.986446,1.050502,19/12/2025 0:11,4,0,11
719,720,0,0,0,1,0,0,0,0.509526,0.986939,0.887088,20/12/2025 11:13,4,11,13
720,721,1,0,0,1,0,0,0,0.595817,0.972845,1.053910,21/12/2025 1:43,4,1,43
721,722,1,0,0,1,0,0,0,0.720618,0.987641,0.895026,22/12/2025 12:12,4,12,12


In [19]:
x_test_submission = np.asarray(test_dataset_df[['3D','symmetric','food','person','architectural','animal','nature','bargain','inflation','grade']])

In [20]:
test_dataset_df = pd.read_csv('Day2/dataset2/NextGen Regression/sample_submission.csv')

In [21]:
print(x_test_submission.shape)

(723, 10)


In [22]:
predictions = []
with torch.no_grad():
    y_test_pred_submission = model(torch.tensor(x_test_submission,dtype=torch.float32)).tolist()
    y_test_pred_submission = [i[0] for i in y_test_pred_submission]

    print(y_test_pred_submission)
        
        
        

[2751.587890625, 1677.734375, 1791.3466796875, 1630.733154296875, 2658.6416015625, 2747.06201171875, 2701.306640625, 2753.57421875, 2765.126708984375, 2689.348876953125, 2746.91748046875, 2786.25732421875, 2765.228271484375, 1675.73583984375, 1755.325927734375, 2761.646728515625, 1669.44970703125, 2754.0791015625, 2619.96142578125, 2745.34619140625, 2766.6689453125, 2195.50146484375, 1689.434326171875, 2740.204833984375, 2682.459228515625, 2783.46142578125, 2769.53369140625, 2668.195068359375, 2636.833984375, 2747.2822265625, 2619.76904296875, 1777.327880859375, 1587.7744140625, 2755.12890625, 2718.08740234375, 2672.752197265625, 2678.986328125, 2745.876953125, 1775.474609375, 1729.60546875, 2635.48046875, 1690.3765869140625, 2744.943603515625, 446.66986083984375, 2695.375, 2690.0712890625, 3456.326171875, 2653.73388671875, 2622.2373046875, 1818.063720703125, 2662.923583984375, 2746.23974609375, 2763.384521484375, 1679.204833984375, 2775.41455078125, 2627.3818359375, 2683.3017578125, 2

In [23]:
test_dataset_df['value']=y_test_pred_submission

In [24]:
test_dataset_df

,ID,value
0,1,2751.587891
1,2,1677.734375
2,3,1791.346680
3,4,1630.733154
4,5,2658.641602
...,...,...
718,719,2750.879883
719,720,1597.396362
720,721,2667.714111
721,722,2650.365967


In [25]:
test_dataset_df.to_csv('Day2/submission/submission.csv',index = False)